# **Benchmark Model**

## Libraries

In [11]:
# Libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from PIL import Image
import uuid
from models.builder import SimpleKeras

from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

from utils.display_utils import show_image
from utils.constants import METADATA_PATH, IMAGE_DIR, IMAGE_SIZE_STANDARD
from utils.preprocess_utils import resize_image

In [12]:
# Setting options
pd.set_option('display.max_rows', None)

## Data Loading

In [13]:
df = pd.read_csv(f'../{METADATA_PATH}')
df.head()

,rare_species_id,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,75fd91cb-2881-41cd-88e6-de451e8b60e2,12853737,449393,animalia,mollusca,unionidae,mollusca_unionidae/12853737_449393_eol-full-si...
1,28c508bc-63ff-4e60-9c8f-1934367e1528,20969394,793083,animalia,chordata,geoemydidae,chordata_geoemydidae/20969394_793083_eol-full-...
2,00372441-588c-4af8-9665-29bee20822c0,28895411,319982,animalia,chordata,cryptobranchidae,chordata_cryptobranchidae/28895411_319982_eol-...
3,29cc6040-6af2-49ee-86ec-ab7d89793828,29658536,45510188,animalia,chordata,turdidae,chordata_turdidae/29658536_45510188_eol-full-s...
4,94004bff-3a33-4758-8125-bf72e6e57eab,21252576,7250886,animalia,chordata,indriidae,chordata_indriidae/21252576_7250886_eol-full-s...


## Data Preprocessing

In [14]:
# Defining the model
model = 'VGG16'

In [15]:
# Resizing the images
resized_tensor = np.array([
    keras.preprocessing.image.smart_resize(
        keras.utils.load_img(f"../{IMAGE_DIR}/{img}")            
        ,IMAGE_SIZE_STANDARD[model]
    )
    for img in df['file_path']
])

c:\Users\Admin\anaconda\envs\ML\Lib\site-packages\PIL\Image.py:3406: DecompressionBombWarning: Image size (115600000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [16]:
resized_tensor

array([[[[1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
         [1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
         [1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
         ...,
         [1.03000000e+02, 9.90000000e+01, 1.00000000e+02],
         [1.03000000e+02, 9.90000000e+01, 1.00000000e+02],
         [1.03000000e+02, 9.80000000e+01, 1.02000000e+02]],

        [[1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
         [1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
         [1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
         ...,
         [1.03000000e+02, 9.90000000e+01, 1.00000000e+02],
         [1.03000000e+02, 9.90000000e+01, 1.00000000e+02],
         [1.03000000e+02, 9.80000000e+01, 1.02000000e+02]],

        [[1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
         [1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
         [1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
         ...,
         [1.03000000e+02, 9.90000000e+01, 1.00000000e+02],
         [

In [17]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

labels = df['phylum'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
num_classes = len(encoder.classes_)
one_hot_labels = to_categorical(encoded_labels, num_classes=num_classes)

# Simple split for testing
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    resized_tensor, one_hot_labels, test_size=0.2, random_state=42
)

In [18]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).shuffle(buffer_size=1024)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) # TODO didnt resolve input_shape yet 
base_model.trainable = False  # Freeze base model

# TODO resolve it and remove it from here, just a placeholder for now
instructions = {
    "input": {"shape": (224, 224, 3)},
    "layers": [
        ("Conv2D", {"filters": 32, "kernel_size": 3, "activation": "relu"}),
        ("MaxPooling2D", {"pool_size": (2, 2)}),
        ("Flatten", {}),
        ("Dense", {"units": 128, "activation": "relu"})
    ],
    "n_classes": num_classes
}

model = SimpleKeras.build_model(instructions, base_model=base_model)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# idk if you want it to put in class
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Evaluate
val_loss, val_accuracy = model.evaluate(val_ds)
print(f"Validation accuracy: {val_accuracy:.4f}")

Epoch 1/10
107/300 ━━━━━━━━━━━━━━━━━━━━ 7:39 2s/step - accuracy: 0.7886 - loss: 3.1270

With single epoch got 0.77 acc runs for a long time